# Homework 5

## Problem 1

SQP

In [13]:
#pip install qpsolvers[open_source_solvers]

import torch as t

#Initializing x
x = t.tensor([[1.], [1.]]) # x[0] = x1 and x[1] = x2

#Initializing mu and W just to get calculation started -- calculated with initial conditions
u = t.tensor([[6.], [2.]]) # u[0] = u1 and u[1] = u2
#W = t.tensor([[2., 0.], [0., (2 + 2*u[0] + 2*u[1])]]) 
w = t.abs(u) #inital weight for line search
alpha = 1 #initial step size

#Calculating Lagrange Gradient
gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  )

#While loop
e = 0.001
while t.norm(gL) > e:
    #Solve for s(k), u(k+1) using QP ************************************************************************************************************
    #: ADD HOW I DID THIS ---------------------------------------------------------------------------------------------------------------------------------------------------- 
    W = t.tensor([[2., 0.], [0., (2 + 2*u[0] + 2*u[1])]]) 
    g = t.tensor([[(x[1]**2 - 2*x[0])], [((x[1] - 1)**2 + 5*x[0] -15)]])
    A = t.tensor([[-2., 2*x[1]], [5., (2*x[1] - 2) ]])
    A_inv = t.inverse(A)
    fx = t.tensor([[2*x[0]], [(2*x[1] - 6)]])
    
    s_dum = t.matmul(-t.transpose(g, 0, 1),A_inv)
    s = t.transpose(s_dum, 0, 1) #s(k)
    
    u_first = fx + t.matmul(W, s)
    u_dum = t.matmul(-t.transpose(u_first, 0, 1),A_inv)
    u = t.transpose(u_dum, 0, 1) #mu(k+1)
    
    #Line search, alpha(k), merit function and armijo line search ********************************************************************************
    w1 = t.abs(u) #weight option 1
    w2 = 0.5*w + 0.5*t.abs(u) #weight option 2
    w = t.maximum(w1, w2) #weight
    Xm = x + alpha*s
    Gm = t.tensor([[(Xm[1]**2 - 2*Xm[0])], [((Xm[1] - 1)**2 + 5*Xm[0] -15)]]) #constraints with x = x + alpha*s
    oB_fun = Xm[0]**2 + (Xm[1] - 3)**2 #objective function with x = x + alpha*s
    mF = oB_fun + w[0]*max(0, Gm[0]) + w[1]*max(0, Gm[1]) #merit function
    
    dg_Dalpha = t.matmul(A, s)
    dmF = t.matmul(t.transpose(fx, 0, 1), s) +  w[0]*max(0, dg_Dalpha[0]) + w[1]*max(0, dg_Dalpha[1]) #derivative of merit function
    phi = mF + alpha*dmF
    
    if mF > phi:
        alpha_dum = alpha
        alpha = alpha_dum*0.5
    
    #x(k+1) = x(k) + alpha(k)*s(k)
    x_dum = x + alpha*s
    x = x_dum
    
    #W(k+1), BFGS
    
    
    #Calculate Lagrange Gradient
    gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  ) #need to update x and mu above
    print(t.norm(gL))
   
objective_fun = oB_fun = x[0]**2 + (x[1] - 3)**2 
print('x1 =', x[0])
print('x2 =', x[1])
print('Objective function value =', objective_fun)



tensor(155.4080)
tensor(32.8042)
tensor(5.8503)
tensor(2.5497)
tensor(2.4888)
tensor(2.4652)
tensor(2.4451)
tensor(2.4336)
tensor(2.4277)
tensor(2.4247)
tensor(2.4232)
tensor(2.4224)
tensor(2.4220)
tensor(2.4218)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(2.4217)
tensor(

KeyboardInterrupt: 

## Problem 2

moon lander